In [12]:
import geopandas as gpd
import pandas as pd

In [13]:
gdf = gpd.read_file("data/provincias.geojson")

In [ ]:
df = pd.DataFrame().replace()

In [17]:
gdf = gdf.replace("Araba","Álava")


In [18]:
gdf.sort_values("codigo")

,codigo,ccaa,provincia,cod_ccaa,texto,geometry
44,01,País Vasco,Álava,16,Álava,"POLYGON ((-3.02679 43.21665, -3.01900 43.21076..."
22,02,Castilla - La Mancha,Albacete,08,Albacete,"POLYGON ((-1.48366 39.41824, -1.48845 39.41739..."
20,03,Comunitat Valenciana,Alacant,10,Alicante,"MULTIPOLYGON (((-0.48546 38.16507, -0.48545 38..."
39,04,Andalucía,Almería,01,Almería,"MULTIPOLYGON (((-2.19104 36.72004, -2.19129 36..."
3,05,Castilla y León,Ávila,07,Ávila,"POLYGON ((-5.01386 41.16425, -5.01094 41.16197..."
13,06,Extremadura,Badajoz,11,Badajoz,"POLYGON ((-4.67789 39.43485, -4.67328 39.42708..."
18,07,Illes Balears,Illes Balears,04,Islas Baleares,"MULTIPOLYGON (((4.09396 40.08836, 4.09450 40.0..."
26,08,Cataluña,Barcelona,09,Barcelona,"MULTIPOLYGON (((1.65707 42.11443, 1.66435 42.1..."
34,09,Castilla y León,Burgos,07,Burgos,"MULTIPOLYGON (((-2.93489 42.59441, -2.93960 42..."
23,10,Extremadura,Cáceres,11,Cáceres,"POLYGON ((-6.23047 40.48660, -6.22392 40.48440..."


In [19]:
def load_turism_data()-> pd.DataFrame:
    #===================================================================
    # Preproceso de datos turísticos


    turistas = pd.read_csv("data/52047.csv",sep=";")
    turistas["year"] = turistas["Periodo"].apply(lambda x: x[:4])
    turistas["Total"] = turistas["Total"].str.replace(".","")
    turistas["Total"] = turistas["Total"].str.replace(",",".")
    turistas["Total"] = turistas["Total"].str.replace(r'^\s*$', "0", regex=True)
    turistas["Total"] = turistas.Total.fillna(0).astype(float)

    turistas["month"] = turistas["Periodo"].apply(lambda x: x[5:])
    turistas["day"] = 1
    turistas["fecha"] = pd.to_datetime(turistas[["month","year","day"]])
    name_fix = {"Alicante/Alacant":"Alacant",
                "Balears, Illes":"Illes Balears",
                "Castellón/Castelló":"Castelló",
                "Coruña, A":"A Coruña",
                "Rioja, La":"La Rioja",
                "Valencia/València":"València",
                "Gipuzkoa":"Gipuzcoa",
                "Araba/Álava":"Araba",
                "Palmas, Las":"Las Palmas"}
    turistas["Provincia de destino"] = turistas["Provincia de destino"].apply(lambda x: name_fix[x] if x in name_fix else x)
    turistas.drop(columns="RESIDENCIA/ORIGEN",inplace=True)

    prov_geo = 'data/provincias.geojson'
    provs = gpd.read_file(prov_geo)
    turistas = turistas.merge(right=provs[["codigo","provincia"]],right_on="provincia",left_on="Provincia de destino",how="left")
    turistas.codigo.fillna("00",inplace=True)
    # viajeros = turistas[turistas["Concepto turístico"] == "Turistas"].groupby(["year","Provincia de destino"]).sum(numeric_only=True).reset_index()
    # viajeros.rename(columns={"Provincia de destino":"NAMEUNIT"},inplace=True)
    # provincias_viajeros = provincias.merge(right=viajeros,on="NAMEUNIT")
    return turistas

In [21]:
df_turismo = load_turism_data()

In [26]:
df_turismo[df_turismo["Países"].isna() & ~df_turismo["Continentes"].isna()]

,Continentes,Países,Provincia de destino,Concepto turístico,Periodo,Total,year,month,day,fecha,codigo,provincia
0,NaN,NaN,Total Nacional,Turistas,2023M03,5639935.0,2023,03,1,2023-03-01,00,NaN
1,NaN,NaN,Total Nacional,Turistas,2023M02,4502738.0,2023,02,1,2023-02-01,00,NaN
2,NaN,NaN,Total Nacional,Turistas,2023M01,4602539.0,2023,01,1,2023-01-01,00,NaN
3,NaN,NaN,Total Nacional,Turistas,2022M12,4451187.0,2022,12,1,2022-12-01,00,NaN
4,NaN,NaN,Total Nacional,Turistas,2022M11,5011551.0,2022,11,1,2022-11-01,00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1395220,Oceanía,NaN,Zaragoza,Duración media de los viajes,2019M11,7.6,2019,11,1,2019-11-01,50,Zaragoza
1395221,Oceanía,NaN,Zaragoza,Duración media de los viajes,2019M10,10.2,2019,10,1,2019-10-01,50,Zaragoza
1395222,Oceanía,NaN,Zaragoza,Duración media de los viajes,2019M09,6.4,2019,09,1,2019-09-01,50,Zaragoza
1395223,Oceanía,NaN,Zaragoza,Duración media de los viajes,2019M08,9.0,2019,08,1,2019-08-01,50,Zaragoza
